In [1]:
import os

import pandas as pd
import numpy as np

In [2]:
leagues = ['EPL', 'Ligue1', 'Bundesliga', 'SerieA', 'LaLiga']

In [3]:
all_dfs = []
for league in leagues:
    cwd = f'dfs/{league}/match_reports'
    if os.path.isdir(cwd):
        onlyfiles = [os.path.join(cwd, f) for f in os.listdir(cwd) if os.path.isfile(os.path.join(cwd, f))]
        
        dfs = []
        for file in onlyfiles:
            df = pd.read_csv(file, parse_dates= ['Date'])
            if 'Match' in df.columns:
                df = df.drop('Match', 1)
            dfs.append(df)

        # for date in dfs[0].Date.unique():
        #     subset_df0 = dfs[0][dfs[0]['Date'] == date]
        #     subset_df1 = dfs[1][dfs[1]['Date'] == date]
        #     if len(subset_df0) != len(subset_df1):
        #         print(date)
        
#         dfs[0] = dfs[0][dfs[0]['Date'] != "2021-04-30"]

        for i in range(6):
            dfs[i].columns = dfs[i].columns.str.replace(' ', '_')
#             print(dfs[i].columns)
            
        new_df = dfs[0].copy()
        for i in range(1, 6):
            new_df = pd.merge(new_df, dfs[i], how='inner', on=['Season', 'League', 'Team', 'Date', 'Player', '#', 'Nation', 'Pos',
               'Age', 'Min'])
        
        all_dfs.append(new_df.reset_index(drop=True))
        
        print(len(dfs[0]), len(dfs[1]), len(new_df))



40541 40541 40541
39120 39092 39092
34095 34095 34095
42219 42219 42219
42225 42194 42194


In [6]:
assert len(all_dfs[0].columns) == len(all_dfs[1].columns)

In [7]:
merged_df = pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)

In [8]:
merged_df.head()

,Season,League,Team,Date,Player,#,Nation,Pos,Age,Min,...,Carries_PrgDist,Carries_Prog_y,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog
0,2020-2021,EPL,Fulham,2020-09-12,Aboubakar Kamara,47,mr MTN,FW,25-189,62.0,...,23.0,2.0,0.0,0.0,3.0,4.0,26.0,18.0,69.2,2.0
1,2020-2021,EPL,Fulham,2020-09-12,Aleksandar Mitrović,9,rs SRB,FW,25-362,28.0,...,3.0,0.0,0.0,0.0,1.0,0.0,12.0,7.0,58.3,1.0
2,2020-2021,EPL,Fulham,2020-09-12,Ivan Cavaleiro,17,pt POR,LW,26-330,90.0,...,76.0,4.0,2.0,0.0,2.0,1.0,41.0,34.0,82.9,1.0
3,2020-2021,EPL,Fulham,2020-09-12,Neeskens Kebano,7,cd COD,RW,28-186,62.0,...,59.0,3.0,1.0,0.0,2.0,0.0,26.0,22.0,84.6,3.0
4,2020-2021,EPL,Fulham,2020-09-12,Andre-Frank Zambo Anguissa,29,cm CMR,RW,24-301,28.0,...,60.0,2.0,0.0,0.0,0.0,2.0,20.0,17.0,85.0,1.0


In [9]:
merged_df = merged_df.drop([
    'Carries_Carries_y', 
    'Carries_Prog_y', 
    'Dribbles_Att_y', 
    'Dribbles_Succ_y', 
    'Performance_CrdR_y', 
    'Performance_CrdY_y', 
    'Performance_Int_y' 
], 1)

merged_df = merged_df.rename(columns = {
    'Carries_Carries_x': 'Carries_Carries', 
    'Carries_Prog_x': 'Carries_Prog', 
    'Dribbles_Att_x': 'Dribbles_Att', 
    'Dribbles_Succ_x': 'Dribbles_Succ', 
    'Performance_CrdR_x': 'Performance_CrdR', 
    'Performance_CrdY_x': 'Performance_CrdY', 
    'Performance_Int_x': 'Performance_Int'
})

In [10]:
import json

with open('dfs/player_match_reports_column_documentation.json') as f:
    col_meanings = json.load(f)

In [11]:
set(merged_df.columns) - set(col_meanings)

set()

In [12]:
set(col_meanings) - set(merged_df.columns)

set()

In [13]:
len(merged_df)

198141

In [15]:
merged_df.to_csv('dfs/player_match_reports.csv', index=False)

In [16]:
merged_df.to_parquet('dfs/player_match_reports.parquet', index=False)

In [17]:
load_df = pd.read_parquet('dfs/player_match_reports.parquet')
load_df

,Season,League,Team,Date,Player,#,Nation,Pos,Age,Min,...,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog
0,2020-2021,EPL,Fulham,2020-09-12,Aboubakar Kamara,47,mr MTN,FW,25-189,62.0,...,45.0,23.0,0.0,0.0,3.0,4.0,26.0,18.0,69.2,2.0
1,2020-2021,EPL,Fulham,2020-09-12,Aleksandar Mitrović,9,rs SRB,FW,25-362,28.0,...,16.0,3.0,0.0,0.0,1.0,0.0,12.0,7.0,58.3,1.0
2,2020-2021,EPL,Fulham,2020-09-12,Ivan Cavaleiro,17,pt POR,LW,26-330,90.0,...,189.0,76.0,2.0,0.0,2.0,1.0,41.0,34.0,82.9,1.0
3,2020-2021,EPL,Fulham,2020-09-12,Neeskens Kebano,7,cd COD,RW,28-186,62.0,...,134.0,59.0,1.0,0.0,2.0,0.0,26.0,22.0,84.6,3.0
4,2020-2021,EPL,Fulham,2020-09-12,Andre-Frank Zambo Anguissa,29,cm CMR,RW,24-301,28.0,...,124.0,60.0,0.0,0.0,0.0,2.0,20.0,17.0,85.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198136,2017-2018,LaLiga,Real Sociedad,2018-05-20,Raúl Navas,22,es ESP,CB,30-009,90.0,...,144.0,63.0,0.0,0.0,0.0,1.0,35.0,34.0,97.1,2.0
198137,2017-2018,LaLiga,Real Sociedad,2018-05-20,Diego Llorente,3,es ESP,CB,24-277,90.0,...,181.0,128.0,0.0,0.0,1.0,0.0,23.0,23.0,100.0,0.0
198138,2017-2018,LaLiga,Real Sociedad,2018-05-20,Aritz Elustondo,15,es ESP,RB,24-053,71.0,...,75.0,54.0,1.0,0.0,0.0,1.0,19.0,15.0,78.9,1.0
198139,2017-2018,LaLiga,Real Sociedad,2018-05-20,Álvaro Odriozola,19,es ESP,RB,22-157,19.0,...,124.0,89.0,2.0,3.0,0.0,1.0,15.0,13.0,86.7,3.0
